SageMaker Studio notebooks provide a set of built-in images for popular data science and ML frameworks and compute options to run notebooks. The built-in SageMaker images contain the Amazon SageMaker Python SDK and the latest version of the backend runtime process, also called kernel. With the custom images feature, you can register custom built images and kernels, and make them available to all users sharing a SageMaker Studio domain. You can start by cloning and extending one of the example Docker files provided by SageMaker, or build your own images from scratch.


You can add a custom **R** image to SageMaker Studio so you can build and train your R models with SageMaker. After attaching the custom R image, you can select the image in Studio and use R to access the SDKs using the **RStudio reticulate** package.

You can create images and image versions and attach image versions to your domain using the SageMaker Studio Control Panel, the AWS SDK for Python (Boto3), and the AWS Command Line Interface (AWS CLI).

In [2]:
import sagemaker

In [3]:
ROLE_ARN = sagemaker.get_execution_role()
ROLE_ARN

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20210411T125966 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


'arn:aws:iam::367158743199:role/service-role/AmazonSageMaker-ExecutionRole-20210411T125966'

In [17]:
# !aws sagemaker describe-notebook-instance --notebook-instance-name 

In [12]:
!aws configure list | grep region | awk '{print $2}'

us-east-2


In [23]:
%%sh

ACCOUNT_ID=$(aws sts get-caller-identity --query Account --output text)
# Get the region defined in the current configuration (default to us-west-2 if none defined)
REGION=$(aws configure list | grep region | awk '{print $2}')


# create a repository in ECR, and then login to ECR repository
aws --region ${REGION} ecr create-repository --repository-name smstudio-custom
aws ecr --region ${REGION} get-login-password | docker login --username AWS \
    --password-stdin ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/smstudio-custom

# Build the docker image and push to Amazon ECR (modify image tags and name as required)
$(aws ecr get-login --region ${REGION} --no-include-email)

# docker build . -t smstudio-r -t ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/smstudio-custom:r 
# docker push ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/smstudio-custom:r
    
# Using with SageMaker Studio
## Create SageMaker Image with the image in ECR (modify image name as required)
# aws sagemaker create-image \
#     --region ${REGION} \
#     --image-name custom-r \
#     --role-arn "arn:aws:iam::367158743199:role/MySagemakerRol"


# aws sagemaker create-image-version \
#     --region ${REGION} \
#     --image-name custom-r \
#     --base-image ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/smstudio-custom:r

## Create AppImageConfig for this image (modify AppImageConfigName and 
## KernelSpecs in app-image-config-input.json as needed)
## note that 'file://' is required in the file path
# aws sagemaker create-app-image-config \
#     --region ${REGION} \
#     --cli-input-json file://app-image-config-input.json


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'smstudio-custom' already exists in the registry with id '367158743199'
sh: 9: docker: not found
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='UTF-8'>
BrokenPipeError: [Errno 32] Broken pipe
sh: 13: docker: not found


CalledProcessError: Command 'b'\nACCOUNT_ID=$(aws sts get-caller-identity --query Account --output text)\n# Get the region defined in the current configuration (default to us-west-2 if none defined)\nREGION=$(aws configure list | grep region | awk \'{print $2}\')\n\n\n# create a repository in ECR, and then login to ECR repository\naws --region ${REGION} ecr create-repository --repository-name smstudio-custom\naws ecr --region ${REGION} get-login-password | docker login --username AWS \\\n    --password-stdin ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/smstudio-custom\n\n# Build the docker image and push to Amazon ECR (modify image tags and name as required)\n$(aws ecr get-login --region ${REGION} --no-include-email)\n\n# docker build . -t smstudio-r -t ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/smstudio-custom:r \n# docker push ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/smstudio-custom:r\n    \n# Using with SageMaker Studio\n## Create SageMaker Image with the image in ECR (modi